In [1]:
import json

def extract_course_prefixes(file_path):
    """
    Reads a JSON file containing a list of course codes, extracts the 
    first 4 letters of each code, and returns them as a list of strings.
    """
    try:
        with open(file_path, 'r') as f:
            course_codes = json.load(f)
        
        # Extract the first 4 letters from each course code
        prefixes = [code[:4] for code in course_codes]
        
        return list(sorted(set(prefixes)))
        
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error: The file '{file_path}' does not contain valid JSON.")
        return []

# Usage example:
# prefixes = extract_course_prefixes('data/course_codes_only.json')
# print(prefixes)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os
import concurrent.futures # <--- Thư viện để chạy đa luồng
import time

MAX_WORKER = 5


# Danh sách các mã ngành quan trọng của EAIT
EAIT_PREFIXES = extract_course_prefixes('../data/course_codes_only.json')

def fetch_course_codes_via_search(prefix):
    # Dùng trang SEARCH thay vì Browse
    url = f"https://my.uq.edu.au/programs-courses/search.html?keywords={prefix}&searchType=course"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    codes = set()
    
    print(f"   -> [Bắt đầu] Đang tìm kiếm: {prefix}...")
    try:
        # Timeout để tránh bị treo nếu mạng lag
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"      ⚠️ Lỗi kết nối {prefix}: {response.status_code}")
            return []

        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)
        
        for link in links:
            href = link['href']
            if "course_code=" in href:
                try:
                    part = href.split("course_code=")[1]
                    code = part.split("&")[0]
                    if code.startswith(prefix) and len(code) == 8:
                        codes.add(code)
                except:
                    continue
        
        print(f"      ✅ [Xong] {prefix}: Tìm thấy {len(codes)} môn.")
        return list(codes)
                    
    except Exception as e:
        print(f"❌ Lỗi khi quét {prefix}: {e}")
        return []

# --- CHẠY LẠI BƯỚC 1 VỚI CONCURRENT ---
def main():
    all_codes = []
    print("🚀 BẮT ĐẦU QUÉT MÃ MÔN (ĐA LUỒNG)...")
    
    start_time = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKER) as executor:
        # Gửi tất cả các task vào executor
        future_to_prefix = {executor.submit(fetch_course_codes_via_search, prefix): prefix for prefix in EAIT_PREFIXES}
        
        # Xử lý kết quả khi từng task hoàn thành
        for future in concurrent.futures.as_completed(future_to_prefix):
            prefix = future_to_prefix[future]
            try:
                data = future.result()
                all_codes.extend(data)
            except Exception as exc:
                print(f'{prefix} generated an exception: {exc}')

    # Lọc trùng và sắp xếp
    all_codes = sorted(list(set(all_codes)))
    
    end_time = time.time()
    
    # Đảm bảo thư mục tồn tại
    output_dir = os.path.join('..', 'data')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'all_course_codes
    .json')

    # Lưu file
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_codes, f, indent=4)

    print(f"\n🎉 TỔNG KẾT: Đã tìm thấy {len(all_codes)} mã môn độc nhất.")
    print(f"⏱️ Thời gian chạy: {end_time - start_time:.2f} giây")
    print("File đã lưu: ", output_path)

if __name__ == "__main__":
    main()

🚀 BẮT ĐẦU QUÉT MÃ MÔN (ĐA LUỒNG)...
   -> [Bắt đầu] Đang tìm kiếm: ACCT...
   -> [Bắt đầu] Đang tìm kiếm: ADPS...
   -> [Bắt đầu] Đang tìm kiếm: AGRC...
   -> [Bắt đầu] Đang tìm kiếm: ANAT...
   -> [Bắt đầu] Đang tìm kiếm: ANIM...
      ✅ [Xong] ADPS: Tìm thấy 11 môn.
   -> [Bắt đầu] Đang tìm kiếm: AUDL...
      ✅ [Xong] ANAT: Tìm thấy 11 môn.
   -> [Bắt đầu] Đang tìm kiếm: AUIL...
      ✅ [Xong] ACCT: Tìm thấy 30 môn.
   -> [Bắt đầu] Đang tìm kiếm: BINF...
      ✅ [Xong] ANIM: Tìm thấy 40 môn.
   -> [Bắt đầu] Đang tìm kiếm: BIOC...
      ✅ [Xong] AGRC: Tìm thấy 56 môn.
   -> [Bắt đầu] Đang tìm kiếm: BIOL...
      ✅ [Xong] AUDL: Tìm thấy 25 môn.
   -> [Bắt đầu] Đang tìm kiếm: BIOM...
      ✅ [Xong] AUIL: Tìm thấy 4 môn.
   -> [Bắt đầu] Đang tìm kiếm: BIOT...
      ✅ [Xong] BINF: Tìm thấy 3 môn.
   -> [Bắt đầu] Đang tìm kiếm: BISM...
      ✅ [Xong] BIOC: Tìm thấy 20 môn.
   -> [Bắt đầu] Đang tìm kiếm: BSAN...
      ✅ [Xong] BIOL: Tìm thấy 67 môn.
   -> [Bắt đầu] Đang tìm kiếm: CHEM...
 